In [1]:
import sys,os

from htmd.ui import *
# from htmd.protocols.equilibration_v2 import Equilibration
import openbabel
import numpy as np
from rdkit import Chem
from rdkit.Chem import rdShapeHelpers
from random import randint

# import Bio.PDB

In [2]:
def obconv(in_type, out_type, in_file, out_file, options=[]):
    obc = openbabel.OBConversion()
    obc.SetInAndOutFormats(in_type, out_type)
    for o in options:
        obc.AddOption(o)
        
    mol = openbabel.OBMol()
    obc.ReadFile(mol, in_file)
    mol.AddHydrogens()
    obc.WriteFile(mol, out_file)
    

def prep_protein(protein_pdb):
    mol = Molecule(protein_pdb)
    mol.filter('protein or not water')
    prot = proteinPrepare(mol)
#     prot.remove('resname WAT')
#     prot.remove('resname DUM')
    preppdb = protein_pdb.replace('.pdb', '_prepared.pdb')
    prot.write(preppdb)
    obconv(in_type='pdb', out_type='pdbqt', in_file=preppdb, out_file=preppdb.replace('.pdb', '.pdbqt'), 
           options=['x', 'r'])
    return prot
    
    
def prep_ligands(lig_sdf):
    obconv(in_type='sdf', out_type='pdbqt', in_file=lig_sdf, out_file=lig_sdf.replace('.sdf', '.pdbqt'), options=['h'])

    
def define_box(ref_lig, output, bufx=10, bufy=10, bufz=10, exhaustiveness=20, seed=None, cpu=4):
    mol = Chem.MolFromMolFile(ref_lig)
    conf = mol.GetConformer()
    params = rdShapeHelpers.ComputeConfBox(conf)
    
    c1 = np.array(params[0])
    c2 = np.array(params[1])
    
    center = np.mean((c1, c2), axis=0)
    
    dims = np.abs(c1-c2)
    
    box_dims = [dims[0] + bufx, dims[1] + bufy, dims[2] + bufz]
    
    if not seed:
        seed = randint(0, 2147483647)
    
    with open(output, 'w') as f:
        f.write(
        """size_x = {}
size_y = {}
size_z = {}
center_x = {}
center_y = {}
center_z = {}
num_modes = 9999
energy_range = 9999
exhaustiveness = {}
cpu={}
seed={}
""".format(box_dims[0], box_dims[1], box_dims[2],
          center[0], center[1], center[2],
          exhaustiveness,
          cpu,
          seed)
        )
        

def solvate_equilibrate(protein, build_dir, output_dir, solv_pad=10, method='amber'):
    prot_solv = solvate(protein, pad=solv_pad)
    if method=='amber':
        amber.build(prot_solv, outdir=build_dir)
    elif method=='charmm':
        charmm.build(prot_solv, outdir=build_dir)
    else:
        raise Exception('please give a method for building!')


def post_md_prep(protein_pdb):
    mol = Molecule(protein_pdb)
    prot = proteinPrepare(mol)
    prot.remove('not protein')
    preppdb = protein_pdb.replace('.pdb', '_prepared.pdb')
    prot.write(preppdb)
    obconv(in_type='pdb', out_type='pdbqt', in_file=preppdb, out_file=preppdb.replace('.pdb', '.pdbqt'), 
           options=['x', 'r'])
    return prot
        

In [12]:
vina_exe = '/dls_sw/apps/xchem/autodock_vina_1_1_2_linux_x86/bin/vina'

In [8]:
apo_prot = '/dls/science/users/uzw12877/KALRNA/XX02KALRNA-x1376_1_apo.pdb'
prot = prep_protein(apo_prot)
prot.remove('resname EDO')
# mol = Molecule(apo_prot)
# mol.filter('protein or not water')

2019-08-05 12:16:26,423 - moleculekit.molecule - INFO - Removed 339 atoms. 2777 atoms remaining in the molecule.
2019-08-05 12:16:28,799 - moleculekit.tools.preparation - WARNING - The following residue has not been optimized: EDO
2019-08-05 12:16:37,358 - moleculekit.tools.preparationdata - INFO - The following residues are in a non-standard state: LYS     5  A (LYN), HIS   103  A (HID), HIS   104  A (HID), HIS  1253  B (HID), HIS  1279  B (HID), HIS  1292  B (HID), HIS  1312  B (HIP), HIS  1341  B (HID), HIS  1353  B (HID), HIS  1410  B (HID)
2019-08-05 12:16:37,360 - moleculekit.tools.preparationdata - WARNING - Dubious protonation state: the pKa of 9 residues is within 1.0 units of pH 7.0.
2019-08-05 12:16:37,361 - moleculekit.tools.preparationdata - WARNING - Dubious protonation state:    LYS     5  A (pKa= 6.68)
2019-08-05 12:16:37,362 - moleculekit.tools.preparationdata - WARNING - Dubious protonation state:    HIS   103  A (pKa= 6.32)
2019-08-05 12:16:37,362 - moleculekit.tools

array([5717, 5718, 5719, 5720], dtype=int32)

In [9]:
os.mkdir('md_amber_build')
os.mkdir('md_amber_run')

In [15]:
solvate_equilibrate(prot, 'md_amber_build/', 'md_amber_run/')

2019-08-05 12:21:26,066 - htmd.builder.solvate - INFO - Using water pdb file at: /dls/science/groups/i04-1/software/anaconda/envs/docking/lib/python3.6/site-packages/htmd/share/solvate/wat.pdb
2019-08-05 12:21:26,705 - htmd.builder.solvate - INFO - Replicating 8 water segments, 2 by 2 by 2
Solvating: 100%|██████████| 8/8 [00:04<00:00,  2.01it/s]
2019-08-05 12:21:32,158 - htmd.builder.solvate - INFO - 12204 water molecules were added to the system.
2019-08-05 12:21:37,232 - htmd.builder.amber - INFO - Starting the build.
2019-08-05 12:21:41,713 - htmd.builder.amber - INFO - Finished building.
2019-08-05 12:21:43,343 - htmd.builder.ionize - INFO - Adding 0 anions + 10 cations for neutralizing and 0 ions for the given salt concentration.
2019-08-05 12:21:46,325 - htmd.builder.builder - WARNING - Segments ['0' '1'] contain both protein and non-protein atoms. Please assign separate segments to them or the build procedure might fail.
2019-08-05 12:21:47,677 - htmd.builder.amber - INFO - Dete

In [47]:
# source /dls/science/groups/i04-1/software/gromacs/bin/GMXRC
# gmx pdb2gmx -f structure.pdb -o structure_processed.gro -water tip4p
# gmx pdb2gmx -f structure.pdb -o structure_processed.gro -water tip4p -ignh -ff amber03
# gmx grompp -f minim.mdp -c structure_processed.gro -p topol.top -o em.tpr
# gmx mdrun -v -deffnm em
# gmx grompp -f nvt.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr
# gmx mdrun -deffnm nvt
# gmx energy -f nvt.edr -o temperature.xvg
# gmx grompp -f npt.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p topol.top -o npt.tpr
# gmx mdrun -deffnm npt
# gmx energy -f npt.edr -o pressure.xvg
# gmx energy -f npt.edr -o density.xvg
# gmx grompp -f md.mdp -c npt.gro -t npt.cpt -p topol.top -o md_0_1.tpr
# gmx mdrun -deffnm md_0_1
# gmx trjconv -s md_0_1.tpr -f md_0_1.xtc -o md_0_1_noPBC.xtc -pbc mol -center
# gmx rms -s md_0_1.tpr -f md_0_1_noPBC.xtc -o rmsd.xvg -tu ns
# gmx rms -s em.tpr -f md_0_1_noPBC.xtc -o rmsd_xtal.xvg -tu ns
# gmx cluster -f md_0_1.xtc -cl

In [46]:
post_md_prep('md_middle_cluster.pdb')

2019-08-05 16:13:19,567 - moleculekit.tools.preparation - WARNING - The following residue has not been optimized: ACE
2019-08-05 16:13:19,568 - moleculekit.tools.preparation - WARNING - The following residue has not been optimized: NME
2019-08-05 16:13:19,568 - moleculekit.tools.preparation - WARNING - The following residue has not been optimized: NA
/dls/science/groups/i04-1/software/anaconda/envs/docking/lib/python3.6/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return getattr(obj, method)(*args, **kwds)
2019-08-05 16:13:23,707 - moleculekit.tools.preparationdata - WARNING - Dubious protonation state: the pKa of 2 residues is within 1.0 units of pH 7.0.
2019-08-05 16:13:23,708 - moleculekit.tools.preparationdata - WARNING - Dubious protonation state:    N+    179T B (pKa= 7.34)
2019-08-05 16:13:23,709 - moleculekit.tools.preparationdata - WARNING - Dubious pr

Molecule with 5713 atoms and 1 frames
Atom field - altloc shape: (5713,)
Atom field - atomtype shape: (5713,)
Atom field - beta shape: (5713,)
Atom field - chain shape: (5713,)
Atom field - charge shape: (5713,)
Atom field - coords shape: (5713, 3, 1)
Atom field - element shape: (5713,)
Atom field - insertion shape: (5713,)
Atom field - masses shape: (5713,)
Atom field - name shape: (5713,)
Atom field - occupancy shape: (5713,)
Atom field - record shape: (5713,)
Atom field - resid shape: (5713,)
Atom field - resname shape: (5713,)
Atom field - segid shape: (5713,)
Atom field - serial shape: (5713,)
angles shape: (0, 3)
bonds shape: (0, 2)
bondtype shape: (0,)
box shape: (3, 1)
boxangles shape: (3, 0)
crystalinfo: None
dihedrals shape: (0, 4)
fileloc shape: (0,)
impropers shape: (0, 4)
reps: 
ssbonds shape: (0,)
step shape: (0,)
time shape: (0,)
viewname: None

In [48]:
os.mkdir('gdp_mdprot_docking')


In [ ]:
define_box('/dls/science/users/uzw12877/KALRNA/gdp_frags/GDP/GDP.sdf', output='gdp_mdprot_docking/GDP_box.txt')

In [54]:
mol = Chem.MolFromMolFile('/dls/science/users/uzw12877/KALRNA/gdp_frags/GDP/GDP.sdf')

In [55]:
print(mol)


In [ ]:
prepare_ligand()

In [2]:
import pymol